In [38]:
import gspread
gc = gspread.service_account(filename='creds/lifts-service.json')
sheet = gc.open_by_key('1XmPko5pbcw0HdvPZeL7Fw8tfX4zhvxREUDGISC54jwk')
ws = sheet.get_worksheet(0)  


In [161]:
import pandas as pd
import altair as alt

alt.renderers.enable('mimetype')

def load_data_LR(L_index: int, R_index: int) -> tuple[str, pd.DataFrame]:
    date = ws.col_values(1)
    L = ws.col_values(L_index)
    R = ws.col_values(R_index)

    title = str(L[0]).split(' ')[0]

    df = pd.DataFrame(
        {
            "date": date[1:],
            "L": L[1:],
            "R": R[1:]
        }
    )

    df["L"] = pd.to_numeric(df["L"])
    df["R"] = pd.to_numeric(df["R"])

    df = df[df['L'] != 0]

    df['date'] = pd.to_datetime(df['date'], dayfirst=True)

    return title, df

def load_data(index: int) -> tuple[str, pd.DataFrame]:
    date = ws.col_values(1)
    val = ws.col_values(index)
    
    title = str(val[0])

    df = pd.DataFrame(
        {
            "date": date[1:],
            title: val[1:]
        }
    )

    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df[title] = pd.to_numeric(df[title])
    df = df[df[title] != 0]

    return title, df

def get_LR_X() -> alt.X:
    return alt.X('date:T', timeUnit='yearmonthdate', sort='-x', title='date', axis= alt.Axis(grid=True))

def get_LR_Y(title: str) -> alt.Y:
    return alt.Y('value:Q', scale=alt.Scale(zero=False), title = title + ' Size (cm)', impute=alt.ImputeParams(frame=[-1, 1], method='mean'))

def LR_transform(df: pd.DataFrame) -> alt.Chart:
    return alt.Chart(df).transform_fold(['L', 'R'])

def make_chart_LR(L_index: int, R_index: int) -> alt.Chart:
    title, df = load_data_LR(L_index, R_index)

    transformed = LR_transform(df)
    chart = transformed.mark_line(point = True).encode(
        get_LR_X(),
        get_LR_Y(title),
        color='key:N'
    ).properties(
        width=1000,
    )

    return chart


def make_chart_LR_hover(L_index: int, R_index: int) -> alt.Chart:
    title, df = load_data_LR(L_index, R_index)

    highlight = alt.selection(type='single', on='mouseover',
                          fields=['key'], nearest=True, empty='none')

    transformed  = LR_transform(df)

    chart = transformed.mark_line().encode(
        get_LR_X(),
        get_LR_Y(title),
        color='key:N',
        size=alt.condition(~highlight, alt.value(1), alt.value(3))
    ).properties(
        width=1000,
    )

    points = transformed.mark_point(opacity=1, filled=True).encode(
        x='date:T',
        y='value:Q',
        color='key:N'
    ).add_selection(
        highlight
    )

    return chart + points

def quant_title(title: str) -> str:
    return title + ':Q'

def get_X() -> alt.X:
    return alt.X('date:T', timeUnit='yearmonthdate', sort='-x', title='date', axis= alt.Axis(grid=False))

def get_Y(title: str, withImpute: bool) -> alt.Y:
    # weight chart throws JS error with impute, not sure why
    if withImpute:
        return alt.Y(quant_title(title), scale=alt.Scale(zero=False), title = title, impute=alt.ImputeParams(frame=[-1, 1], method='mean'))
    else:
        return alt.Y(quant_title(title), scale=alt.Scale(zero=False), title = title)

def chart_style() -> dict:
    return {
        'background':'#161619', 'axis': (alt.Axis(labelColor='#666666', titleColor='#ffffff', gridColor='#131316'))
    }

def make_chart(index: int, withImpute: bool = True) -> alt.Chart:
    title, df = load_data(index)
    chart = alt.Chart(df).mark_line(point = True).encode(
        get_X(),
        get_Y(title, withImpute) ,
        color=alt.value('#e9702b')
    ).properties(
        width=1500,
        height=250
    ).configure(**chart_style())

    return chart

def make_chart_with_tool_tip(index: int, withImpute: bool = True) -> alt.Chart:
    title, df = load_data(index)

    chart = alt.Chart(df).mark_line(point = True).encode(
        get_X(),
        get_Y(title, withImpute) ,
        tooltip = ['date', title]
    ).properties(
        width=1500,
        height=250
    )

    return chart

def make_chart_with_hover_tool_tip(index: int, withImpute: bool = True) -> alt.Chart:

    title, df = load_data(index)
    title_q = quant_title(title)

    nearest = alt.selection(type='single', nearest=True, on='mouseover', fields=['date'], empty='none')

    line = alt.Chart(df).mark_line().encode(
        get_X(),
        get_Y(title, withImpute) ,
        tooltip = ['date', title]
    )

    selectors = alt.Chart(df).mark_point().encode(
        x=get_X(),
        opacity=alt.value(0),
    ).add_selection(
        nearest
    )

    points = line.mark_point().encode(
        opacity=alt.condition(nearest, alt.value(1), alt.value(0))
    )

    text = line.mark_text(align='left', dx=5, dy=-5).encode(
        text=alt.condition(nearest, title_q, alt.value(' '))
    )

    rules = alt.Chart(df).mark_rule(color='gray').encode(
        x=get_X(),
    ).transform_filter(nearest)

    chart = alt.layer(
        line, selectors, points, rules, text
    ).properties(
        width=600, height=300
    )

    return chart



In [112]:
# weight graph (lower is better)

chart = make_chart_with_tool_tip(4, False)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [162]:
# Hips
chart = make_chart(9)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [114]:
# Waist (lower is better)
chart = make_chart_with_hover_tool_tip(10)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [115]:
# Chest

chart = make_chart_with_tool_tip(11)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [116]:
# Calve Graph
chart = make_chart_LR_hover(5, 6)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [117]:
# Thighs
chart = make_chart_LR(7,8)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [118]:
# Forearms
chart = make_chart_LR(12,13)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html


In [119]:
# Biceps
chart = make_chart_LR_hover(14, 15)

chart

<VegaLite 4 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/troubleshooting.html
